<a href="https://colab.research.google.com/github/sazio/MultiAgentCovid/blob/master/Multiplex_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Data and match Agents with Buildings

In [65]:
import numpy as np
import pandas as pd 
import glob 

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [4]:
path = "drive/MyDrive/MAS/Data/Output_GAMA/Data"

agents_path = path + "/Agents"
buildings_path = path + "/Buildings"

agents_files = glob.glob(agents_path + "/*.csv")
buildings_file = glob.glob(buildings_path + "/*.csv")

In [137]:
DF_buildings = pd.read_csv(buildings_file[0], sep = ",", names = ["name", "loc_x", "loc_y", "loc_z"])
DF_buildings = DF_buildings.iloc[1:]
DF_buildings = DF_buildings.reset_index(drop = True)
DF_buildings["name"] = DF_buildings["name"].str[:8] + "(" + DF_buildings["name"].str[8:] + ")"
DF_buildings["loc_x"] = DF_buildings["loc_x"].str[1:].astype(np.float32)
DF_buildings["loc_y"] = DF_buildings["loc_y"].astype(np.float32)
DF_buildings["loc_z"] = DF_buildings["loc_z"].str[:-1].astype(np.float32)

DF_buildings.head()

,name,loc_x,loc_y,loc_z
0,building(0),4258.587891,8622.923828,0.0
1,building(1),4205.688477,8554.613281,0.0
2,building(2),4172.957031,8508.589844,0.0
3,building(3),4653.243164,8230.745117,0.0
4,building(4),4630.689941,8173.111816,0.0


In [138]:
# First Agents file as example (ToDo --> function from this stuff)
DF_agents = pd.read_csv(agents_files[0], sep = ",", names = ["name", "cycle","loc_x", "loc_y", "loc_z","living_place",
"working_place","stuff_place", "is_susceptible", "is_infected", "is_immune"])
DF_agents = DF_agents.iloc[1:]
DF_agents = DF_agents.reset_index(drop = True)
DF_agents["loc_x"] = DF_agents["loc_x"].str[1:].astype(np.float32)
DF_agents["loc_y"] = DF_agents["loc_y"].astype(np.float32)
DF_agents["loc_z"] = DF_agents["loc_z"].str[:-1].astype(np.float32)

DF_agents = DF_agents.replace({"is_susceptible":{"true":True, "false":False}})
DF_agents = DF_agents.replace({"is_infected":{"True":True, "False":False}})
DF_agents = DF_agents.replace({"is_immune":{"True":True, "False":False}})

DF_agents.head()

,name,cycle,loc_x,loc_y,loc_z,living_place,working_place,stuff_place,is_susceptible,is_infected,is_immune
0,people0,401,4047.333740,2014.397461,0.0,building(1408),building(220),building(3381),False,False,True
1,people1,401,1911.168823,3917.855225,0.0,building(1350),building(5862),building(6186),False,True,False
2,people2,401,3277.526123,657.434082,0.0,building(2112),building(84),building(6398),True,False,False
3,people3,401,3803.600098,1790.961182,0.0,building(5496),building(5860),building(6211),True,False,False
4,people4,401,3402.263672,7576.838379,0.0,building(1101),building(266),building(2286),True,False,False


In [139]:
np.linalg.norm(DF_agents.values[0,2:4] - DF_buildings.values[0,1:3])

6611.902075943781

In [165]:
x = DF_buildings[DF_buildings.name == DF_agents.values[2,5]].values[0][1:3]
y = DF_agents.values[2,2:4]
np.linalg.norm(x-y)

3164.990054408083

In [166]:
def is_in_which_building(DF_buildings, DF_agents, agent_index):
  
  """
  Check if the agent is in some building or in the street, useful to build a multiplex network 

  """

  living_coord = DF_buildings[DF_buildings.name == DF_agents.values[agent_index,5]].values[0][1:3]
  working_coord = DF_buildings[DF_buildings.name == DF_agents.values[agent_index,6]].values[0][1:3]
  stuff_coord = DF_buildings[DF_buildings.name == DF_agents.values[agent_index,7]].values[0][1:3]

  agent_coord = DF_agents.values[agent_index,2:4]

  liv_ag_dist = np.linalg.norm(living_coord - agent_coord)
  wor_ag_dist = np.linalg.norm(working_coord - agent_coord)
  stu_ag_dist = np.linalg.norm(stuff_coord - agent_coord)



  if min(liv_ag_dist, wor_ag_dist, stu_ag_dist) == liv_ag_dist and liv_ag_dist < 80:

    return DF_agents.values[agent_index,5]

  elif  min(liv_ag_dist, wor_ag_dist, stu_ag_dist) == wor_ag_dist and wor_ag_dist < 80:

    return DF_agents.values[agent_index,6]

  elif  min(liv_ag_dist, wor_ag_dist, stu_ag_dist) == stu_ag_dist and stu_ag_dist < 80:

    return DF_agents.values[agent_index,7]

  else: 

    return "None" 


In [167]:
is_in_which_building(DF_buildings, DF_agents, agent_index = 3)

'building(6211)'